# __ETL__ _(Extract, Transform, Load)_

## Introducción

Este notebook se enfoca en el proceso de **ETL** utilizando datos extraídos de las plataformas Yelp y Google Maps. Este proceso implica una _extracccion,transformación y carga_ de los datos con el objetivo de prepararlos para análisis posteriores. Este paso es crucial en cualquier proyecto de ciencia de datos para garantizar la calidad y utilidad de los datos.

## Configuraciones Globales e Importaciones

En esta sección, se instalan e importan todas las librerías y/o módulos necesarios para el proceso ETL (Extract, Transform, Load) y se establecen configuraciones globales de ser requerido. Se utilizan las siguientes librerías y herramientas:

In [ ]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalamos SPARK para manejar grande volumnes de datos

In [ ]:
#Instala pyspark en Google Colaboratory
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=a658c847d0d02dbded35e6d7d61a5e8105bdf38522f7aa3a9a7c498743cb0e11
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Importamos librerias necesarias

In [ ]:
import warnings
warnings.filterwarnings("ignore") # Se utiliza para gestionar las advertencias y mantener el código limpio.

In [ ]:
import re
import os # Proporciona funciones para interactuar con el sistema operativo.
import pandas as pd # Una librería de análisis de datos.
import seaborn as sns #S e utiliza para la visualización de datos.
import json # Se utiliza para trabajar con datos en formato JSON.

# YELP

**Dataset:** BUSSINES

DECLARACIÓN DE LA RUTA DE LOS DATA SET

In [ ]:
# Ruta local completa del archivo business.pkl
ruta_business = "/content/drive/MyDrive/Colab-Notebooks/Yelp/business.pkl"

In [ ]:
# Cargar el archivo pickle usando pd.read_pickle
df_bussi = pd.read_pickle(ruta_business)

In [ ]:
# Mostramos las primeras 5 filas del dataframe
df_bussi.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Verficaciones de CANTIDAD DE FILAS**

In [ ]:
#Contamos la contidad de filas de nuestro DATAFRAME BUSSINES
df_bussi.count()

business_id     150346
name            150346
address         150346
city            150346
state           150343
postal_code     150346
latitude        150346
longitude       150346
stars           150346
review_count    150346
is_open         150346
attributes      136602
categories      150243
hours           127123
business_id          5
name                 5
address              5
city                 5
state                5
postal_code          5
latitude             5
longitude            5
stars                5
review_count         5
is_open              5
attributes           5
categories           5
hours                5
dtype: int64

### Buscamos los **DUPLICADOS**

In [ ]:
# Verificación de columnas duplicadas en el DataFrame business_yelp
columns = df_bussi.columns.duplicated()
columns

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [ ]:
# Filtrar el DataFrame business_yelp para eliminar columnas duplicadas
df_bussi = df_bussi.loc[:, ~columns]
df_bussi.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [ ]:
#Contamos la contidad de filas de nuestro DATAFRAME BUSSINES
df_bussi.count()#Contamos la contidad de filas de nuestro DATAFRAME BUSSINES

business_id     150346
name            150346
address         150346
city            150346
state           150343
postal_code     150346
latitude        150346
longitude       150346
stars           150346
review_count    150346
is_open         150346
attributes      136602
categories      150243
hours           127123
dtype: int64

In [ ]:
# Reemplazamos los NaN en 'state' por sus valores correspondientes de acuerdo a 'address', 'city' y 'postal_code'
df_bussi.at[0, 'state'] = 'CA'
df_bussi.at[1, 'state'] = 'MO'
df_bussi.at[2, 'state'] = 'AZ'

### **Buscamos los NULOS**

In [ ]:
def contar_nulos(dataframe):

    # Obtener la cantidad de valores nulos por columna
  nulos_por_columna = dataframe.isnull().sum()

  print("Cantidad de valores nulos por columna:\n", nulos_por_columna)

In [ ]:
#Cuantos nulos hay en cada columna
contar_nulos(df_bussi)

Cantidad de valores nulos por columna:
 business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64


In [ ]:
#Eliminamos los NULOS en 'attributes'
df_bussi = df_bussi.dropna(subset=['attributes'])

In [ ]:
#Eliminamos los NULOS en 'hours'
df_bussi = df_bussi.dropna(subset=['hours'])

In [ ]:
#Cuantos nulos hay en cada columna
contar_nulos(df_bussi)

Cantidad de valores nulos por columna:
 business_id     0
name            0
address         0
city            0
state           0
postal_code     0
latitude        0
longitude       0
stars           0
review_count    0
is_open         0
attributes      0
categories      0
hours           0
dtype: int64


Informacion de los tipos de datos en nuestro **DATAFRAME**

In [ ]:
df_bussi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117618 entries, 1 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   117618 non-null  object
 1   name          117618 non-null  object
 2   address       117618 non-null  object
 3   city          117618 non-null  object
 4   state         117618 non-null  object
 5   postal_code   117618 non-null  object
 6   latitude      117618 non-null  object
 7   longitude     117618 non-null  object
 8   stars         117618 non-null  object
 9   review_count  117618 non-null  object
 10  is_open       117618 non-null  object
 11  attributes    117618 non-null  object
 12  categories    117618 non-null  object
 13  hours         117618 non-null  object
dtypes: object(14)
memory usage: 13.5+ MB


**Filtramos en la columna CATEGORIES - RESTAURANTES**

In [ ]:
# Filtrar filas donde la columna 'CATEGORIES' contiene 'restaurant' en cualquier parte del texto
df_bussi = df_bussi[df_bussi['categories'].str.contains(r'restaurant', case=False, regex=True, na=False)]

# Mostrar los resultados
print(df_bussi)


                   business_id                           name  \
3       MTSW4McQd7CbVtyjqoe9mw             St Honore Pastries   
5       CF33F8-E6oudUQ46HnavjQ                 Sonic Drive-In   
9       bBDDEgkFA1Otx9Lfe7BZUQ                 Sonic Drive-In   
11      eEOYSgkmpB90uNA7lDOMRA          Vietnamese Food Truck   
12      il_Ro8jwPlHresjw9EGmBg                        Denny's   
...                        ...                            ...   
150325  l9eLGG9ZKpLJzboZq-9LRQ                           Wawa   
150327  cM6V90ExQD6KMSU3rRB5ZA              Dutch Bros Coffee   
150336  WnT9NIzQgLlILjPT0kEcsQ  Adelita Taqueria & Restaurant   
150339  2O2K6SXPWv56amqxCECd4w                   The Plum Pit   
150340  hn9Toz3s-Ei3uZPt7esExA          West Side Kebab House   

                      address             city state postal_code   latitude  \
3                 935 Race St     Philadelphia    CA       19107  39.955505   
5               615 S Main St     Ashland City    AZ       37

In [ ]:
df_bussi = df_bussi[df_bussi['state'].isin(['NY', 'CA', 'WI'])]

In [ ]:
df_bussi.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
29,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,3001 Highway 31 W,White House,CA,37188,36.464747,-86.659187,3.5,8,1,{'RestaurantsPriceRange2': '1'},"Pizza, Chicken Wings, Sandwiches, Restaurants","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ..."
88,LhpPSrulqVeTyJeK2xydvQ,Fresh Fruits & Salads,114 N 3rd St,Camden,CA,08102,39.94669,-75.123327,4.5,6,1,"{'BusinessParking': '{'garage': False, 'street...","Juice Bars & Smoothies, Restaurants, Fruits & ...","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
222,8c0r7olQSYGcws0bTd3ikw,Zesty Tsunami,6614 Osteen Rd,New Port Richey,CA,34653,28.250218,-82.682441,4.0,5,0,"{'RestaurantsReservations': 'False', 'Restaura...","Hawaiian, Food, Food Trucks, Street Vendors, R...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
409,QWqKTWQ2OiDgo3dzNkpung,Jong Ka Jib,6600 N 5th St,Philadelphia,CA,19126,40.052064,-75.127394,4.5,351,1,"{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...","Korean, Restaurants","{'Monday': '10:30-22:30', 'Tuesday': '10:30-22..."
618,3uvFklejzRPwEPEZQ5vHYQ,Wawa,100 E Old Baltimore Pike,Media,CA,19063,39.916176,-75.387655,3.5,14,1,"{'Alcohol': 'u'none'', 'RestaurantsReservation...","Delis, Convenience Stores, Restaurants, Food, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
694,jsQQu9rVerR3OGxIa0hRQQ,Kelly's Taproom,1107 Lancaster Ave,Bryn Mawr,CA,19010,40.024588,-75.324239,3.0,63,1,"{'RestaurantsReservations': 'False', 'Restaura...","Bars, Beer Bar, American (Traditional), Nightl...","{'Tuesday': '16:0-2:0', 'Wednesday': '16:0-2:0..."
821,JomDoQafikclSVVCPkWjnA,Cedar Park Cafe,4914 Baltimore Ave,Philadelphia,CA,19143,39.94796,-75.22198,4.0,301,1,"{'WiFi': 'u'no'', 'OutdoorSeating': 'False', '...","Diners, Restaurants, Sandwiches, Breakfast & B...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
850,wMyHVb54Ne9SNasa4kjOCg,Carnivore,5257 Shaw Ave,St. Louis,CA,63110,38.618412,-90.274182,4.5,281,1,"{'HasTV': 'True', 'RestaurantsPriceRange2': '2...","Restaurants, Breakfast & Brunch, Steakhouses, ...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:0', ..."
881,Wc-c_hQK4vFCl9keeVTUZw,Felix's Restaurant & Oyster Bar,7400 Lakeshore Dr,New Orleans,CA,70124,30.023792,-90.113622,3.5,255,1,"{'Caters': 'True', 'RestaurantsAttire': 'u'cas...","Restaurants, Cajun/Creole, Seafood, Southern","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."


### **Eliminamos las columnas con las que no iremos a trabajar**

In [ ]:
#Contamos la contidad de filas de nuestro DATAFRAME BUSSINES
df_bussi.count()

business_id     1554
name            1554
address         1554
city            1554
state           1554
postal_code     1554
latitude        1554
longitude       1554
stars           1554
review_count    1554
is_open         1554
attributes      1554
categories      1554
hours           1554
dtype: int64

In [ ]:
# Eliminamos'is_open', 'postal_code'.
df_bussi = df_bussi.drop(columns=['is_open', 'postal_code'])
df_bussi.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'latitude',
       'longitude', 'stars', 'review_count', 'attributes', 'categories',
       'hours'],
      dtype='object')

In [ ]:
df_bussi.head(10)

,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
29,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,3001 Highway 31 W,White House,CA,36.464747,-86.659187,3.5,8,{'RestaurantsPriceRange2': '1'},"Pizza, Chicken Wings, Sandwiches, Restaurants","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ..."
88,LhpPSrulqVeTyJeK2xydvQ,Fresh Fruits & Salads,114 N 3rd St,Camden,CA,39.94669,-75.123327,4.5,6,"{'BusinessParking': '{'garage': False, 'street...","Juice Bars & Smoothies, Restaurants, Fruits & ...","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
222,8c0r7olQSYGcws0bTd3ikw,Zesty Tsunami,6614 Osteen Rd,New Port Richey,CA,28.250218,-82.682441,4.0,5,"{'RestaurantsReservations': 'False', 'Restaura...","Hawaiian, Food, Food Trucks, Street Vendors, R...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
409,QWqKTWQ2OiDgo3dzNkpung,Jong Ka Jib,6600 N 5th St,Philadelphia,CA,40.052064,-75.127394,4.5,351,"{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...","Korean, Restaurants","{'Monday': '10:30-22:30', 'Tuesday': '10:30-22..."
618,3uvFklejzRPwEPEZQ5vHYQ,Wawa,100 E Old Baltimore Pike,Media,CA,39.916176,-75.387655,3.5,14,"{'Alcohol': 'u'none'', 'RestaurantsReservation...","Delis, Convenience Stores, Restaurants, Food, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
694,jsQQu9rVerR3OGxIa0hRQQ,Kelly's Taproom,1107 Lancaster Ave,Bryn Mawr,CA,40.024588,-75.324239,3.0,63,"{'RestaurantsReservations': 'False', 'Restaura...","Bars, Beer Bar, American (Traditional), Nightl...","{'Tuesday': '16:0-2:0', 'Wednesday': '16:0-2:0..."
821,JomDoQafikclSVVCPkWjnA,Cedar Park Cafe,4914 Baltimore Ave,Philadelphia,CA,39.94796,-75.22198,4.0,301,"{'WiFi': 'u'no'', 'OutdoorSeating': 'False', '...","Diners, Restaurants, Sandwiches, Breakfast & B...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
850,wMyHVb54Ne9SNasa4kjOCg,Carnivore,5257 Shaw Ave,St. Louis,CA,38.618412,-90.274182,4.5,281,"{'HasTV': 'True', 'RestaurantsPriceRange2': '2...","Restaurants, Breakfast & Brunch, Steakhouses, ...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:0', ..."
881,Wc-c_hQK4vFCl9keeVTUZw,Felix's Restaurant & Oyster Bar,7400 Lakeshore Dr,New Orleans,CA,30.023792,-90.113622,3.5,255,"{'Caters': 'True', 'RestaurantsAttire': 'u'cas...","Restaurants, Cajun/Creole, Seafood, Southern","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."


### **MANIPULAMOS LA COLUMNA LATITUD Y LONGITUD**

In [ ]:
# Obtener la cantidad de decimales en la columna de latitud
df_bussi['lat_decimal_count'] = df_bussi['latitude'].apply(lambda x: len(str(x).split('.')[-1]))

# Obtener la cantidad de decimales en la columna de longitud
df_bussi['lon_decimal_count'] = df_bussi['longitude'].apply(lambda x: len(str(x).split('.')[-1]))

In [ ]:
# Verificar si todas las filas tienen la misma cantidad de decimales en latitud
lat_decimal_same = df_bussi['lat_decimal_count'].nunique() == 1

# Verificar si todas las filas tienen la misma cantidad de decimales en longitud
lon_decimal_same = df_bussi['lon_decimal_count'].nunique() == 1

In [ ]:
# Mostrar el resultado
print("Latitud tiene la misma cantidad de decimales en todas las filas:", lat_decimal_same)
print("Longitud tiene la misma cantidad de decimales en todas las filas:", lon_decimal_same)

# Eliminar las columnas auxiliares
df_bussi.drop(['lat_decimal_count', 'lon_decimal_count'], axis=1, inplace=True)

Latitud tiene la misma cantidad de decimales en todas las filas: False
Longitud tiene la misma cantidad de decimales en todas las filas: False


In [ ]:
# Función para completar con ceros a la derecha hasta llegar a la cantidad deseada de decimales
def completar_decimales(valor, decimales):
    partes = str(valor).split('.')
    if len(partes) == 1:
        return f"{valor}.{'0' * decimales}"
    elif len(partes[1]) < decimales:
        return f"{valor}{'0' * (decimales - len(partes[1]))}"
    else:
        return valor

# Número de decimales deseado
decimales = 6

# Redondear latitud y longitud a la cantidad deseada de decimales
df_bussi['latitude'] = df_bussi['latitude'].astype(float).round(decimales)
df_bussi['longitude'] = df_bussi['longitude'].astype(float).round(decimales)

# Convertir de nuevo a string y completar con ceros a la derecha si tienen menos de 6 decimales
df_bussi['latitude'] = df_bussi['latitude'].apply(lambda x: completar_decimales(x, decimales))
df_bussi['longitude'] = df_bussi['longitude'].apply(lambda x: completar_decimales(x, decimales))

# Verificar si todas las filas tienen la misma cantidad de decimales
latitud_decimales_iguales = df_bussi['latitude'].apply(lambda x: len(str(x).split('.')[-1])) == decimales
longitud_decimales_iguales = df_bussi['longitude'].apply(lambda x: len(str(x).split('.')[-1])) == decimales

print("Latitud tiene la misma cantidad de decimales en todas las filas:", latitud_decimales_iguales.all())
print("Longitud tiene la misma cantidad de decimales en todas las filas:", longitud_decimales_iguales.all())

Latitud tiene la misma cantidad de decimales en todas las filas: True
Longitud tiene la misma cantidad de decimales en todas las filas: True


**ARREGLAMOS LA COLUMNA CATEGORIA**

In [ ]:
# Eliminar filas con valores NaN en la columna 'categories'
df_bussi = df_bussi.dropna(subset=['categories'])

In [ ]:
# Filtrar las filas donde la columna 'categories' contiene la palabra 'Restaurants'
restaurants_df = df_bussi[df_bussi['categories'].str.contains('Restaurants')]

In [ ]:
#Contamos la contidad de filas de nuestro DATAFRAME BUSSINES
df_bussi.count()

business_id     1554
name            1554
address         1554
city            1554
state           1554
latitude        1554
longitude       1554
stars           1554
review_count    1554
attributes      1554
categories      1554
hours           1554
dtype: int64

In [ ]:
df_bussi.head(10)

,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
29,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,3001 Highway 31 W,White House,CA,36.464747,-86.659187,3.5,8,{'RestaurantsPriceRange2': '1'},"Pizza, Chicken Wings, Sandwiches, Restaurants","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ..."
88,LhpPSrulqVeTyJeK2xydvQ,Fresh Fruits & Salads,114 N 3rd St,Camden,CA,39.946690,-75.123327,4.5,6,"{'BusinessParking': '{'garage': False, 'street...","Juice Bars & Smoothies, Restaurants, Fruits & ...","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
222,8c0r7olQSYGcws0bTd3ikw,Zesty Tsunami,6614 Osteen Rd,New Port Richey,CA,28.250218,-82.682441,4.0,5,"{'RestaurantsReservations': 'False', 'Restaura...","Hawaiian, Food, Food Trucks, Street Vendors, R...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
409,QWqKTWQ2OiDgo3dzNkpung,Jong Ka Jib,6600 N 5th St,Philadelphia,CA,40.052064,-75.127394,4.5,351,"{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...","Korean, Restaurants","{'Monday': '10:30-22:30', 'Tuesday': '10:30-22..."
618,3uvFklejzRPwEPEZQ5vHYQ,Wawa,100 E Old Baltimore Pike,Media,CA,39.916176,-75.387655,3.5,14,"{'Alcohol': 'u'none'', 'RestaurantsReservation...","Delis, Convenience Stores, Restaurants, Food, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
694,jsQQu9rVerR3OGxIa0hRQQ,Kelly's Taproom,1107 Lancaster Ave,Bryn Mawr,CA,40.024588,-75.324239,3.0,63,"{'RestaurantsReservations': 'False', 'Restaura...","Bars, Beer Bar, American (Traditional), Nightl...","{'Tuesday': '16:0-2:0', 'Wednesday': '16:0-2:0..."
821,JomDoQafikclSVVCPkWjnA,Cedar Park Cafe,4914 Baltimore Ave,Philadelphia,CA,39.947960,-75.221980,4.0,301,"{'WiFi': 'u'no'', 'OutdoorSeating': 'False', '...","Diners, Restaurants, Sandwiches, Breakfast & B...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
850,wMyHVb54Ne9SNasa4kjOCg,Carnivore,5257 Shaw Ave,St. Louis,CA,38.618412,-90.274182,4.5,281,"{'HasTV': 'True', 'RestaurantsPriceRange2': '2...","Restaurants, Breakfast & Brunch, Steakhouses, ...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:0', ..."
881,Wc-c_hQK4vFCl9keeVTUZw,Felix's Restaurant & Oyster Bar,7400 Lakeshore Dr,New Orleans,CA,30.023792,-90.113622,3.5,255,"{'Caters': 'True', 'RestaurantsAttire': 'u'cas...","Restaurants, Cajun/Creole, Seafood, Southern","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."


## Carga de nuestro archivo

In [35]:
file_path = "/content/drive/MyDrive/Colab-Notebooks/transformaciones/bussi-limpio.csv"
df_bussi.to_csv(file_path, header=True)